In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
# from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pickle
# from sklearn.metrics import confusion_matrix, classification_report
# from sklearn.tree import export_graphviz
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import tree
# import pydotplus
# import collections

# import networkx as nx
# from networkx.drawing.nx_pydot import *
from MVDD.MVDD import MVDD
import MVDD.MVDD_Generator as mvGen
# import networkx as nx
# from networkx.drawing.nx_pydot import *
# import Params as params
# import pandas as pd
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# import numpy as np
# import random
# from itertools import permutations
# import re

plt.rc('font', size=12)
warnings.filterwarnings('ignore')

# Hemo

In [2]:
#Load original dataframes
hemoData = pd.read_csv('Data/Preprocessed Data/Cluster_Hemo.csv', index_col='DEIDNUM')
allScores = hemoData['Score']
death = hemoData['ScoreDeath']
rehosp = hemoData['ScoreRehosp']
readm = hemoData['ScoreReadmission']
hemoData

,RAP,PAS,PAD,PAMN,PCWP,PCWPMod,PCWPA,PCWPMN,CO,CI,...,PAPP,SVR,RAT,PPRatio,Age,EjF,Score,ScoreDeath,ScoreRehosp,ScoreReadmission
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
72,24.0,42.0,24.0,30.0,36.0,36.0,36.0,36.0,2.20,1.60,...,0.428571,5357.575758,0.666667,0.266667,88.0,25.0,5,5,5,4
72,18.0,42.0,24.0,30.0,24.0,24.0,24.0,28.0,3.10,2.00,...,0.428571,3509.677419,0.750000,0.272727,88.0,25.0,3,3,4,2
81,10.0,40.0,20.0,27.0,18.0,18.0,18.0,20.0,4.52,2.02,...,0.500000,1982.300885,0.555556,0.242857,69.0,20.0,1,2,1,1
81,12.0,35.0,15.0,25.0,17.0,17.0,17.0,14.0,4.65,2.08,...,0.571429,1772.043011,0.705882,0.571429,69.0,20.0,1,2,1,1
814,3.0,51.0,19.0,30.0,18.0,18.0,18.0,18.0,4.10,2.00,...,0.627451,2621.138211,0.166667,0.566667,58.0,NaN,3,3,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,57.0,30.0,4,4,3,5
99302,19.0,87.0,37.0,54.0,44.0,44.0,44.0,NaN,3.90,2.00,...,0.574713,2666.666667,0.431818,0.642857,41.0,15.0,5,5,5,4
99302,11.0,70.0,28.0,44.0,28.0,28.0,28.0,NaN,5.30,2.70,...,0.600000,2440.251572,0.392857,1.088608,41.0,15.0,3,3,4,2


In [3]:
# Preprocess and create training and testing sets
hemo = hemoData.drop('Score', axis=1)
hemo = hemo.drop('ScoreDeath', axis=1)
hemo = hemo.drop('ScoreRehosp', axis=1)
hemo = hemo.drop('ScoreReadmission', axis=1)
hemo = hemo.replace(np.inf, 0)
hemo = hemo.fillna(0)
xTrain, xTest, yTrain, yTest = train_test_split(hemo, allScores, test_size=.2)

print(xTrain.shape, xTest.shape)

(334, 27) (84, 27)


## All Scores Label

In [4]:
#Perform Grid search to find best parameter set
trialParams = {'criterion': ('gini', 'entropy'), 'max_depth': [3, 5, 7, 10, 13, 15, 20, 25, 30, None], 
               'min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30]}

mvGen.findBestModelParams(xTrain, yTrain, trialParams)

Best parameters set found on training set:
{'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1}
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        38
           2       1.00      1.00      1.00       108
           3       1.00      1.00      1.00       111
           4       1.00      1.00      1.00        29
           5       1.00      1.00      1.00        48

    accuracy                           1.00       334
   macro avg       1.00      1.00      1.00       334
weighted avg       1.00      1.00      1.00       334



In [6]:
#Train model using best params

#Generate 5 trees
for t in range(5):
    mvdd = mvGen.generateTree(xData=xTrain, yData=yTrain, classes=["1", "2", "3", "4", "5"], learningCriteria='gini', 
                   maxLevels=None, minSamplesPerLeaf=1, modelName='TreeFiles/Hemo_AllOutcomes'+t)
    
    #print accuracy for each generated tree
    acc = mvdd.predictScoreSet(xData=xTest, yData=yTest, confusionMatrix=False, report=True)

              precision    recall  f1-score   support

           1       1.00      0.43      0.60         7
           2       0.86      0.86      0.86        28
           3       0.85      0.88      0.86        32
           4       0.83      1.00      0.91         5
           5       0.79      0.92      0.85        12

    accuracy                           0.85        84
   macro avg       0.86      0.82      0.81        84
weighted avg       0.85      0.85      0.84        84



In [ ]:
## All Data